In [7]:
import numpy as np
import nltk
import io

In [11]:
def get_reviews(path, positive=True):
    label = 1 if positive else 0
    
    with io.open(path, encoding='latin-1') as f:
        review_text = f.readlines()
        
    reviews = []
    for text in review_text:
        # return a tuple of the review text and a label for whether it is a positive or negative review
        reviews.append((text, label))
    return reviews

In [9]:
def extract_reviews():
    positive_reviews = get_reviews("rt-polarity.pos", positive=True)
    negative_reviews = get_reviews("rt-polarity.neg", positive=False)
    
    return positive_reviews, negative_reviews

In [12]:
positive_reviews, negative_reviews = extract_reviews()

In [17]:
TRAIN_DATA = 5000
TOTAL_DATA = len(positive_reviews)

train_reviews = positive_reviews[:TRAIN_DATA] + negative_reviews[:TRAIN_DATA]

test_positive_reviews = positive_reviews[TRAIN_DATA:TOTAL_DATA]
test_negative_reviews = negative_reviews[TRAIN_DATA:TOTAL_DATA]

In [24]:
def get_vocabulary(train_reviews):
    words_set = set()
    
    for review in train_reviews:
        words_set.update(review[0].split())
        
    return list(words_set)

vocabulary = get_vocabulary(train_reviews)

In [27]:
def extract_features(review_text):
    
    # split the review into words and create a set of the words
    review_words = set(review_text.split())
    
    features = {}
    for word in vocabulary:
        features[word] = (word in review_words)
        
    return features

In [32]:
train_features = nltk.classify.apply_features(extract_features, train_reviews)

In [33]:
trained_classifier = nltk.NaiveBayesClassifier.train(train_features)

In [34]:
def sentiment_calculator(review_text):
    features = extract_features(review_text)
    return trained_classifier.classify(features)

In [42]:
sentiment_calculator("What an amazing film")

1

In [44]:
sentiment_calculator("that was such a crap night")

0

In [45]:
def classify_test_reviews(test_positive_reviews, test_negative_reviews, sentiment_calculator):
    
    positive_results = [sentiment_calculator(review[0]) for review in test_positive_reviews]
    negative_results = [sentiment_calculator(review[0]) for review in test_negative_reviews]
    
    true_positives = sum(x > 0 for x in positive_results)
    true_negatives = sum(x == 0 for x in negative_results)
    
    percent_true_positive = float(true_positives) / len(positive_results)
    percent_true_negative = float(true_negatives) / len(negative_results)
    
    total_accurate = true_positives + true_negatives
    total = len(positive_results) + len(negative_results)
    
    print("Accuracy on positive reviews = " + "%.2f" % (percent_true_positive * 100) + "%")
    print("Accuracy on negative reviews = " + "%.2f" % (percent_true_negative * 100) + "%")
    print("Overall accuracy = " + "%.2f" % (total_accurate * 100 / total) + "%")

In [46]:
classify_test_reviews(test_positive_reviews, test_negative_reviews, sentiment_calculator)

Accuracy on positive reviews = 78.25%
Accuracy on negative reviews = 80.66%
Overall accuracy = 79.46%
